# Data Download , Unzip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# COCO Dataset wget images ( train/val/test )
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip

# COCO Dataset wget annotations ( train/val/test )
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget http://images.cocodataset.org/annotations/image_info_test2017.zip


In [ ]:
!mkdir /content/data

In [ ]:
!unzip /content/annotations_trainval2017.zip -d /content/data

!unzip /content/test2017.zip -d /content/data
!unzip /content/train2017.zip -d /content/data
!unzip /content/val2017.zip -d /content/data

unzip:  cannot find or open /content/annotations_trainval2017.zip, /content/annotations_trainval2017.zip.zip or /content/annotations_trainval2017.zip.ZIP.
unzip:  cannot find or open /content/test2017.zip, /content/test2017.zip.zip or /content/test2017.zip.ZIP.
unzip:  cannot find or open /content/train2017.zip, /content/train2017.zip.zip or /content/train2017.zip.ZIP.
unzip:  cannot find or open /content/val2017.zip, /content/val2017.zip.zip or /content/val2017.zip.ZIP.


# Detectron2 Install

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 14474, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 14474 (delta 18), reused 32 (delta 13), pack-reused 14425
Receiving objects: 100% (14474/14474), 5.93 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (10463/10463), done.


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
#!pip install -U torch==1.8 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -U torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:13:47tcmalloc: large alloc 1147494400 bytes == 0x39656000 @  0x7efe4b0f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:11:59tcmalloc: large alloc 1434370048 bytes == 0x7dcac000 @  0x7efe4b0f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.3 MB/s eta 0:08:03tcmalloc: large alloc 1792966656 bytes == 0x2ade000 @  0x7efe4b0f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x5

In [ ]:
# install detectron2:
!pip install detectron2==0.6 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.8/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.8/index.html
     |████████████████████████████████| 6.8 MB 56.1 MB/s 
     |████████████████████████████████| 151 kB 29.7 MB/s 
     |████████████████████████████████| 130 kB 58.8 MB/s 
     |████████████████████████████████| 106 kB 25.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 843 kB 58.7 MB/s 
     |████████████████████████████████| 117 kB 73.6 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=5dc9b6cb20b5017b7d128a59d88c13e9797ed8a82a3fb6ec7d30a99f69ddf66f
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11

# Detectron2 Model, Dataset Setting



In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.config import get_cfg
import random
from detectron2.utils.visualizer import Visualizer
import os

In [ ]:
# json 및 Image path setting
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_dataset_train", {}, "/content/data/annotations/instances_train2017.json", "/content/data/train2017")
register_coco_instances("my_dataset_val", {}, "/content/data/annotations/instances_val2017.json", "/content/data/val2017")
# register_coco_instances("my_dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")


In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

# Train

In [ ]:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 1000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05




cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80 
cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Inference Run

## Inpating setting

In [ ]:
%cd /content
# 깃허브 repository 가져오기
!git clone https://github.com/saic-mdal/lama.git

# 라이브러리 설치
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet
!pip install torchtext==0.9

# 경로이동
%cd /content/lama

# weight 다운로드
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip

# 라이브러리 설치
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

# 라이브러리 호출
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil



canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

# 그림 그리는 함수 선언
def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

## pretrain weight down (detectron2)

In [ ]:
# https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
%cd /content
!wget https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl

/content
--2022-07-29 06:08:42--  https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/139653917/model_final_2d9806.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431414189 (411M) [application/octet-stream]
Saving to: ‘model_final_2d9806.pkl’

model_final_2d9806. 100%[===================>] 411.43M  17.0MB/s    in 25s     

2022-07-29 06:09:08 (16.2 MB/s) - ‘model_final_2d9806.pkl’ saved [431414189/431414189]



## cropper

In [ ]:
import numpy as np
import cv2
from PIL import Image
from matplotlib.image import imread

def cropper(org_image_path, mask_array, out_file_name):
    num_instances = mask_array.shape[0]
    mask_array = np.moveaxis(mask_array, 0, -1)
    mask_array_instance = []
    img = imread(str(org_image_path))
    output = np.zeros_like(img)
    for i in range(num_instances):
        # a = mask_array[:, :, i:(i+1)]
        mask_array_instance.append(mask_array[:, :, i:(i+1)])
        output = np.where(mask_array_instance[i] == True, 255, output)

    im = Image.fromarray(output)
    im.save(out_file_name)

## Inference cfg

In [ ]:
!mkdir /content/data/test2017
!mkdir /content/data/output

In [ ]:
# image /content/data/test2017/file 넣기

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = '/content/data/output/'

cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "/content/model_final_2d9806.pkl"

cfg.DATASETS.TEST = ("my_dataset_test", )

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")


# Detector Segmentation

## coco dic

In [ ]:
coco_class = {
 '0': 'person',
 '1': 'bicycle',
 '10': 'fire hydrant',
 '11': 'stop sign',
 '12': 'parking meter',
 '13': 'bench',
 '14': 'bird',
 '15': 'cat',
 '16': 'dog',
 '17': 'horse',
 '18': 'sheep',
 '19': 'cow',
 '2': 'car',
 '20': 'elephant',
 '21': 'bear',
 '22': 'zebra',
 '23': 'giraffe',
 '24': 'backpack',
 '25': 'umbrella',
 '26': 'handbag',
 '27': 'tie',
 '28': 'suitcase',
 '29': 'frisbee',
 '3': 'motorbike',
 '30': 'skis',
 '31': 'snowboard',
 '32': 'sports ball',
 '33': 'kite',
 '34': 'baseball bat',
 '35': 'baseball glove',
 '36': 'skateboard',
 '37': 'surfboard',
 '38': 'tennis racket',
 '39': 'bottle',
 '4': 'aeroplane',
 '40': 'wine glass',
 '41': 'cup',
 '42': 'fork',
 '43': 'knife',
 '44': 'spoon',
 '45': 'bowl',
 '46': 'banana',
 '47': 'apple',
 '48': 'sandwich',
 '49': 'orange',
 '5': 'bus',
 '50': 'broccoli',
 '51': 'carrot',
 '52': 'hot dog',
 '53': 'pizza',
 '54': 'donut',
 '55': 'cake',
 '56': 'chair',
 '57': 'sofa',
 '58': 'pottedplant',
 '59': 'bed',
 '6': 'train',
 '60': 'diningtable',
 '61': 'toilet',
 '62': 'tvmonitor',
 '63': 'laptop',
 '64': 'mouse',
 '65': 'remote',
 '66': 'keyboard',
 '67': 'cell phone',
 '68': 'microwave',
 '69': 'oven',
 '7': 'truck',
 '70': 'toaster',
 '71': 'sink',
 '72': 'refrigerator',
 '73': 'book',
 '74': 'clock',
 '75': 'vase',
 '76': 'scissors',
 '77': 'teddy bear',
 '78': 'hair drier',
 '79': 'toothbrush',
 '8': 'boat',
 '9': 'traffic light'

}
len(coco_class.keys())

80

In [ ]:
!mkdir /content/lama/data_for_prediction

## inspector inspector_class

In [ ]:
def inspector(select):
    
    while 1 : 
        if len(select) == 0 :
            select = input('공백 또는 입력하신 값이 없습니다. 다시 입력해주세요. : ')
            continue


        b = re.sub('[0-9\s]', '', select)
        if len(b) != 0 :
        
            select = input('숫자 공백을 제외한 문자와 같이 입력되었습니다. 다시 입력 해주세요 : ')

        elif len(b) == 0 :
            select = list(map(int, select.split()))

            if len(boxes)-1 >= max(select) :
                break

            select = input('입력하신 번호는 선택하실 수 있는 번호 내에 없습니다. 다시 입력 해주세요 : ')

    return select

def inspector_class(select):
    
    while 1 : 
        if len(select) == 0 :
            select = input('공백 또는 입력하신 값이 없습니다. 다시 입력해주세요. : ')
            continue


        b = re.sub('[0-9\s]', '', select)
        if len(b) != 0 :
        
            select = input('숫자 공백을 제외한 문자와 같이 입력되었습니다. 다시 입력 해주세요 : ')

        elif len(b) == 0 :
            select = list(map(int, select.split()))
            cnt = 0
            for aa in select :
                if aa in class_cate :
                    continue
                cnt += 1

            if cnt != 0 :
                print('')
                print('번호를 찾을 수 없습니다. 번호를 확인하시고 띄어쓰기를 기준으로 입력해주세요.')
                select = input('선택하시는 종류의 번호를 다시 선택해주세요 : ')
            
            else : break
                    
    return select


In [ ]:
# !rm -rf /content/data

# !mkdir -p /content/data/output
# !mkdir -p /content/data/test2017

In [ ]:
!unzip /content/drive/MyDrive/장기프로젝트/koreandataset/videoplayback.zip -d /content

In [ ]:
for a in os.listdir('/content/content/videoplayback') :
    shutil.move('/content/content/videoplayback/'+a, '/content/data/test2017')

!rm -rf /content/content

In [ ]:
import time
import re
from tqdm import tqdm
##########################
coco_dic = coco_class
##########################

img_list = sorted(os.listdir('/content/data/test2017/'))[:400]

for imageName in tqdm(img_list):
    im = cv2.imread('/content/data/test2017/' + imageName)
    
    h, w, _ = im.shape

    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=1,
                # instance_mode=ColorMode.IMAGE_BW # 굳이 필욘 없네 # remove the colors of unsegmented pixels. This option is only available for segmentation models
                    )

    idxofClass = [i for i, x in enumerate(list(outputs['instances'].pred_classes))] #if x == 0]

    o = outputs["instances"]
    
    boxes = o.pred_boxes[idxofClass]
    masks = o.pred_masks[idxofClass]

    obj = detectron2.structures.instances.Instances(image_size=()) #w,h인지 h,w인지 모름


    obj.set('pred_boxes', boxes)
    obj.set('pred_masks', masks)


    out = v.draw_instance_predictions(obj.to("cpu"))

    boxes = v._convert_boxes(outputs["instances"].pred_boxes.to('cpu'))#.squeeze()

    for idx, box in enumerate(boxes):
        pred_class_num = outputs['instances'][idx].pred_classes.tolist()[0]
        class_text = coco_dic[str(pred_class_num)]
        
        out = v.draw_text(f"{idx} : {class_text}", ((box[0]+box[2])/2, (box[1]+box[3])/2), font_size = h//45, color = 'g') # 0번부터 아니고 1번부터 시작으로 할 수 있을 듯 (사용자 편의)

    # cv2_imshow(out.get_image()[:, :, ::-1])

    # # 사진이 print되는데 시간이 걸려서 대기시간 줌
    # time.sleep(3)

#############
    mask = outputs['instances'].pred_masks

    class_cate = outputs['instances'].to('cpu').pred_classes.tolist()
    class_cate = list(set(class_cate))

    # print('------선택지-------')
    # for c in class_cate :
        # print(f'{c} : {coco_dic[str(c)]}')
    
    # print('-------------------')
    # print('')

    # select = input('')

    # select = inspector_class(select)

    # print('')
    # if len(select) == 0 :
        # print('수정할 물체가 없습니다.')
        # continue
    # print('')



    tensor_list = []
    aaaa = [2, 4, 5, 7, 8]
    for idx in range(len(outputs['instances'])) :
        # if outputs['instances'][idx].pred_classes.tolist()[0] in aaaa :
        #     tensor_list.append(mask[idx])
        tensor_list.append(mask[idx])

    if len(tensor_list) >0 :
        tensor_stack = torch.stack(tensor_list)
    else :
        print('바꿀내용이없습니다')
        continue


    img_root = '/content/data/test2017/'
    img_path = img_root + imageName
    out_root = '/content/data/output/'
    out_path = out_root + imageName

    mask_array = tensor_stack.cpu().numpy()

    cropper(img_path, mask_array, out_path)

    shutil.copy(img_path, '/content/lama/data_for_prediction/'+imageName.split('.')[0]+'.jpg')
    # for dilate
    mask = cv2.imread(out_path)
    size = cv2.getStructuringElement(cv2.MORPH_RECT, (60, 60)) # (10, 10) 클 수록 팽창 범위 증가
    dilate_img = cv2.dilate(mask, size)
    
    cv2.imwrite('/content/lama/data_for_prediction/'+imageName.split('.')[0] +'_mask.png', dilate_img)

 

100%|██████████| 400/400 [06:24<00:00,  1.04it/s]


# Inpainting Run

In [ ]:
%cd /content/lama

import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (15,5)
# plt.rcParams['figure.dpi'] = 200
# plt.subplot(131)

!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output dataset.img_suffix=.jpg > /dev/null


# for a in os.listdir('/content/output')[:-4] :
#   # 출력데이터도 png
#   plt.imshow(plt.imread(f"/content/output/"+a))
#   _=plt.axis('off')
#   _=plt.title('inpainting result')
#   plt.show()
#   fname = None

/content/lama


In [ ]:
# import cv2
# import numpy as np
# import glob

# frameSize = (1920, 1080)

# out = cv2.VideoWriter('/content/test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 29, frameSize)

# for filename in tqdm(sorted(glob.glob('/content/output/*.png'))):
#     img = cv2.imread(filename)
#     out.write(img)

# out.release()

100%|██████████| 794/794 [00:54<00:00, 14.67it/s]


In [ ]:
# shutil.copy('/content/test.mp4', '/content/drive/MyDrive/공유폴더/inpatingvideo.mp4')

'/content/drive/MyDrive/공유폴더/inpatingvideo.mp4'

In [ ]:
# from IPython.display import HTML
# from base64 import b64encode
 
# def show_video(video_path, video_width = 600):
   
#   video_file = open(video_path, "r+b").read()
 
#   video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
#   return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
# show_video('/content/test.mp4')

영상 inpainting

In [ ]:
!mkdir -p /content/videoinpainting/jpg
!mkdir -p /content/videoinpainting/mask

In [ ]:
for aa in os.listdir('/content/lama/data_for_prediction') :
    if aa[-3:] == 'jpg' :
        shutil.copy('/content/lama/data_for_prediction/'+aa , '/content/videoinpainting/jpg')
    if aa[-3:] == 'png' :
        shutil.copy('/content/lama/data_for_prediction/'+aa, '/content/videoinpainting/mask/'+aa[:-9]+'.jpg')


In [ ]:
import shutil

In [ ]:
%cd /content
!git clone https://github.com/ruiliu-ai/FuseFormer.git

%cd FuseFormer
!pip install -r requirements.txt
!mkdir /content/FuseFormer/checkpoints
shutil.copy('/content/drive/MyDrive/장기프로젝트/koreandataset/fuseformer.pth', '/content/FuseFormer/checkpoints/')

/content
Cloning into 'FuseFormer'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 62 (delta 24), reused 50 (delta 12), pack-reused 0
Unpacking objects: 100% (62/62), done.
/content/FuseFormer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 37.6 MB/s 
     |████████████████████████████████| 26.3 MB 1.2 MB/s 
     |████████████████████████████████| 2.2 MB 42.1 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.17.2
    Uninstalling scikit-image-0.17.2:
      Successfully uninstalled scikit-image-0.17.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

'/content/FuseFormer/checkpoints/fuseformer.pth'

In [ ]:
%cd /content/FuseFormer

/content/FuseFormer


In [ ]:
!python test.py -c /content/FuseFormer/checkpoints/fuseformer.pth -v /content/videoinpainting/jpg -m /content/videoinpainting/mask --width 864 --height 480 --outw 864 --outh 480

loading from: /content/FuseFormer/checkpoints/fuseformer.pth
loading videos and masks from: /content/videoinpainting/jpg
0 6 3
Traceback (most recent call last):
  File "test.py", line 160, in <module>
    main_worker()
  File "test.py", line 135, in main_worker
    pred_img = model(masked_imgs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/content/FuseFormer/model/fuseformer.py", line 158, in forward
    trans_feat = self.transformer(trans_feat)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py", line 119, in forward
    input = module(input)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/content/FuseFormer/

In [ ]:
for a in sorted(os.listdir('/content/videoinpainting/jpg'))[40:] : 
    os.remove('/content/videoinpainting/jpg/'+a)
    
for a in sorted(os.listdir('/content/videoinpainting/mask'))[40:] : 
    os.remove('/content/videoinpainting/mask/'+a)
